 <img src="../code/Resources/cropped-SummerWorkshop_Header.png"> 

<h1 align="center">Homework worksheet 1: Tutorial on neuronal encoding and behavior</h1> 
<h3 align="center">Summer Workshop on the Dynamic Brain</h3> 
<h3 align="center">Thursday, August 26th, 2025</h3> 
<h4 align="center">Day 2</h4> 

In [ ]:
import os
import numpy as np
import numpy.random as npr
import pandas as pd
import pynwb

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

pd.set_option('display.max_columns', None)

%matplotlib inline

# HMM libraries 
import jax.numpy as jnp
import jax.random as jr

from dynamax.hidden_markov_model import GaussianHMM

### Question: Decoding performance depend on behavior states?  

In [ ]:
# pick a session_id and get session data
session_id = '759434_2025-02-04'
nwb_path = f'/root/capsule/data/{session_id}/{session_id}.nwb'
session = pynwb.NWBHDF5IO(nwb_path).read()

In [ ]:
# access trials table and set interval 
trials = session.trials.to_dataframe()
quiescent_start = trials.stim_start_time.values - 1.5
quiescent_stop = trials.quiescent_stop_time.values

trial_start = trials.start_time.values 
trial_stop = trials.stop_time.values 

In [ ]:
def get_trialwise_values(x, timestamps, start, stop, mean_value=True):
    """
    Extracts trial-wise summary statistics of different behaviors (mean or median) from a time-aligned signal.

    Parameters:
    - x : array-like
        Signal values (e.g., neural data or behavioral measurements).
    - timestamps : array-like
        Time points corresponding to each value in x.
    - start : array-like
        Start times for each trial.
    - stop : array-like
        Stop times for each trial.
    - mean_value : bool, default=True
        If True, compute the mean within each trial window; otherwise compute the median.

    Returns:
    - values : list
        List of mean or median values for each trial window.
    """
    
    if mean_value:
        # Compute mean of x within each [start, stop] window
        return [np.nanmean(x[np.logical_and(s1 <= timestamps, timestamps <= s2)]) 
                for s1, s2 in zip(start, stop)]
    else:
        # Compute median of x within each [start, stop] window
        return [np.nanmedian(x[np.logical_and(s1 <= timestamps, timestamps <= s2)]) 
                for s1, s2 in zip(start, stop)]

In [ ]:
# face expressions

facial_data = {}

# facial expressions: 
def get_facial_feature(part_name, facial_features_df):
    confidence = facial_features_df[f'{part_name}_likelihood']
    temporal_norm = facial_features_df[f'{part_name}_temporal_norm']
    x = facial_features_df[f'{part_name}_x']
    y = 492 - facial_features_df[f'{part_name}_y']
    xy = np.sqrt(x**2 + y**2)
    xy[(confidence < 0.98) | (temporal_norm > np.nanmean(temporal_norm) + 3 * np.nanstd(temporal_norm))] = np.nan
    xy = pd.Series(xy).interpolate(limit_direction='both').to_numpy() 
    return xy


facial_features_df = session.processing['behavior']['lp_side_camera'][:]
feature_timestamps = facial_features_df['timestamps'].values.astype('float')
map_names = {'ears': 'ear_base_l', 'jaw': 'jaw', 'nose': 'nose_tip', 'whisker_pad': 'whisker_pad_l_side'}

ear = get_facial_feature('ear_base_l', facial_features_df)
facial_data['ear_median_position'] = get_trialwise_values(ear, feature_timestamps, quiescent_start, quiescent_stop, mean_value=False)

jaw = get_facial_feature('jaw', facial_features_df)
facial_data['jaw_median_position'] =  get_trialwise_values(jaw, feature_timestamps, quiescent_start, quiescent_stop, mean_value=False)

nose = get_facial_feature('nose_tip', facial_features_df)
facial_data['nose_median_position'] = get_trialwise_values(nose, feature_timestamps, quiescent_start, quiescent_stop, mean_value=False)

whisker_pad =  get_facial_feature('whisker_pad_l_side', facial_features_df)
facial_data['whisker_median_position'] = get_trialwise_values(whisker_pad, feature_timestamps,  quiescent_start, quiescent_stop, mean_value=False)

facial_data = pd.DataFrame(facial_data)